## Predication with Different Classification Method to The Dataset

In [ ]:
%pip install scikeras

In [86]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf

from sklearn.exceptions import ConvergenceWarning
import warnings

from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from scikeras.wrappers import KerasClassifier

# Model Selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Ensemble Methods
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import IsolationForest

In [87]:
# Suppress warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [88]:
# Set maximum output lines before scrolling
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.instance().display_formatter.formatters['text/plain'].for_type(
    type, lambda obj, p, cycle: p.text(repr(obj)[:10000])
)

<function __main__.<lambda>(obj, p, cycle)>

### Metrics Calculations

In [89]:
def calculate_metrics(classifier, y_value, y_pred):
    print(f"\n{classifier} Metrics: ")
    print(classification_report(y_value, y_pred))

In [90]:
def fit_model_and_generate_metrics(model, label, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    calculate_metrics(label, y_test, model.predict(X_test))

In [102]:
class ModelTuningAndEvaluation:

    param_grid_logistic_regression = {
        'C': [0.01, 1, 10],
        'solver': ['lbfgs', 'liblinear', 'saga'],
        'penalty': ['l2'],
        'max_iter': [100, 500]
    }

    param_grid_decission_tree_classifier = {
        'max_depth': [None, 5, 20, 50],
        'min_samples_split': [2, 5, 10, 20],
        'criterion': ['gini', 'entropy'],
    }

    param_grid_random_forest_classifier = {
        'n_estimators': [50, 100],
        'max_depth': [10, 20],
        'bootstrap': [True],
        'criterion': ['gini', 'entropy']
    }

    param_grid_gaussian_naive_bias = {
        'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
    }

    param_grid_svc = {
        'C': [0.1, 1, 10, 100, 1000],
        'gamma': [1, 0.1, 0.01, 0.001],
        'kernel': ['rbf', 'poly']
    }

    param_grid_knn = {
        'n_neighbors': [100, 500, 700, 900, 1100],
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski'],
        'p': [1, 2]
    }

    param_grid_ada_boost = {
        'n_estimators': [50, 100],
        'learning_rate': [0.5, 1.0],
        'estimator': [
            DecisionTreeClassifier(max_depth=1)
        ],
    }

    param_grid_xgb = {
        'n_estimators': [50, 100],
        'learning_rate': [0.1, 0.2],
        'subsample': [0.8, 1.0],
        'gamma': [0, 0.1],
    }

    param_grid_grad_boost = {
      'n_estimators': [50, 100],
      'learning_rate': [0.1, 0.2],
      'max_depth': [3, 5],
      'random_state': [42]
    }

    param_grid_ann = {
        'model__n_neurons': [64],
        'model__activation': ['relu', 'tanh'],
        'epochs': [100, 150],
        'batch_size': [50, 100]
    }

    def __init__(self, file_path):
        self.feature_path = file_path
        self.feature_df = self.get_feture()
        self.X, self.y = self.split_feture_and_target()
        # for xgaboost mapping y to start from zero
        self.y = self.map_zero_to_n() 
        self.number_of_categories = self.get_number_of_categories()
        self.X_train, self.X_test, self.y_train, self.y_test = self.get_scale_and_test_train_split()

    # data read and processing section
    def remove_outliear(self, feature_df):
        iso = IsolationForest(contamination=0.01, random_state=42)
        outliers = iso.fit_predict(feature_df)
        data_cleaned = feature_df[outliers == 1]
        return data_cleaned

    def get_feture(self):
        feature_df = pd.read_csv(self.feature_path)
        feature_df = feature_df.iloc[:, 1:] # remove index
        return self.remove_outliear(feature_df)

    def split_feture_and_target(self):
        X = self.feature_df.iloc[:, :-1]
        y = self.feature_df.iloc[:, -1]
        return X, y

    def get_scale_and_test_train_split(self):
        #Scaling
        scaler = StandardScaler()
        scaled_fature = scaler.fit_transform(self.X)
        #test train split
        return train_test_split(scaled_fature, self.y, train_size=.20, random_state=42, stratify=self.y)

    def map_zero_to_n(self):
        unique_values = {val: idx for idx, val in enumerate(self.y.unique())}
        y_mapped = self.y.map(unique_values)
        return y_mapped

    def get_number_of_categories(self):
        return len(self.y.unique())

    def onehot_encode(self):
        self.y_train = to_categorical(self.y_train, num_classes = self.number_of_categories)
        print(self.y_train.shape)

    # Cross validation
    def kfold_cross_validation(self, model, n_splits):
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        kfold_score = cross_val_score(model, self.X, self.y, cv=kf)
        mean_score = np.mean(kfold_score)
        print("\nK-fold cross-validation scores:", kfold_score)
        print("Mean K-fold cross-validation score:", mean_score)

    def stratified_cross_validation(self, model, n_splits):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        skfold_score = cross_val_score(model, self.X, self.y, cv=skf)
        
        mean_score = np.mean(skfold_score)
        print("\nStraified cross validation scores:", skfold_score)
        print("Mean Straified cross-validation score:", mean_score)

    def cross_validation(self, model, n_splits):
        self.kfold_cross_validation(model, n_splits)
        self.stratified_cross_validation(model, n_splits)

    # Hyper-parameter tuning

    def gridSerach(self, estimator, param_grid):
        print("#---------- Grid Search ------------#")

        grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=3, verbose=0)
        grid_search.fit(self.X_train, self.y_train)

        print("Best parameters: ", grid_search.best_params_)
        print("Best score: ", grid_search.best_score_)
        return grid_search

    def randomSearch(self, estimator, param_grid):
        print("\n#---------- Random Search -----------#")

        random_search = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, n_iter=500, cv=3, random_state=42)
        random_search.fit(self.X_train, self.y_train)

        print("Best parameters: ", random_search.best_params_)
        print("Best score: ", random_search.best_score_)
        return random_search

    def hyper_parameter_tuning(self, model, param_grid):
        grid_search = self.gridSerach(model, param_grid)
        random_search = self.randomSearch(model, param_grid)
        return grid_search if grid_search.best_score_ > random_search.best_score_ else random_search

    # Models section
    def logistic_regression_model(self):
        print("#------------------- #1. Logistic Regression Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(LogisticRegression(), self.param_grid_logistic_regression)
        lrm = tuned_model.best_estimator_

        fit_model_and_generate_metrics(lrm, "Logistic Regression", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(lrm, 3)

    def decission_tree_classifier_model(self):
        print("#-------------------- #2. Decission Tree Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(DecisionTreeClassifier(), self.param_grid_decission_tree_classifier)
        dt = tuned_model.best_estimator_

        fit_model_and_generate_metrics(dt, "Decission Tree Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(dt, 3)

    def random_forest_classifier_model(self):
        print("#-------------------- #3. Random Forest Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(RandomForestClassifier(), self.param_grid_random_forest_classifier)
        rfc = tuned_model.best_estimator_

        fit_model_and_generate_metrics(rfc, "Random Forest Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(rfc, 3)

    def gaussian_naive_bias_classifier_model(self):
        print("#-------------------- #4. Gaussian Naive Bias Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(GaussianNB(), self.param_grid_gaussian_naive_bias)
        gnb = tuned_model.best_estimator_

        fit_model_and_generate_metrics(gnb, "Gaussian Naive Bias Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(gnb, 3)


    def support_vector_classifier_model(self):
        print("#-------------------- #5. Support Vector Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(SVC(), self.param_grid_svc)
        svc = tuned_model.best_estimator_

        fit_model_and_generate_metrics(svc, "Support Vector Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(svc, 3)


    def knn_classifier_model(self):
        print("#-------------------- #6. K-Nearest Neighbors Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(KNeighborsClassifier(), self.param_grid_knn)
        knn = tuned_model.best_estimator_

        fit_model_and_generate_metrics(knn, "K-Nearest Neighbors", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(knn, 3)

    def ada_boost_classifier_model(self):
        print("#-------------------- #7. Ada-Boost Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(AdaBoostClassifier(), self.param_grid_ada_boost)
        abc = tuned_model.best_estimator_

        fit_model_and_generate_metrics(abc, "Ada-Boost Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(abc, 3)

    def xg_boost_classifier_model(self):
        print("-------------------- #8. XG Boost Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(XGBClassifier(), self.param_grid_xgb)
        xgb = tuned_model.best_estimator_

        fit_model_and_generate_metrics(xgb, "XG Boost Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(xgb, 3)

    def gradient_boost_model(self):
        print("#-------------------- #9. Gradient Boost Classifier Model --------------------#")

        tuned_model = self.hyper_parameter_tuning(GradientBoostingClassifier(), self.param_grid_grad_boost)
        gb = tuned_model.best_estimator_

        fit_model_and_generate_metrics(gb, "Gradient Boost Classifier", self.X_train, self.X_test, self.y_train, self.y_test)
        self.cross_validation(gb, 3)

    
    # ----------------------- #10. Artificial Neural Net Model ------------------------#
    def ann_kfold_cross_validation(self, model, n_splits=2, epochs=50, batch_size=100):
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
      scores = []

      X = self.X
      y = self.y
      for train_index, val_index in kf.split(X):
          X_train, X_val = X.iloc[train_index], X.iloc[val_index]
          y_train, y_val = y.iloc[train_index], y.iloc[val_index]

          y_train = to_categorical(y_train, num_classes=self.number_of_categories)

          model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

          # Evaluate the model
          y_val_pred = np.argmax(model.predict(X_val), axis=1)  # Convert probabilities to class labels
          accuracy = accuracy_score(y_val, y_val_pred)
          scores.append(accuracy)

      print("\nK-fold cross-validation scores:", scores)
      print("Average score:", np.mean(scores))

    def ann_stratified_cross_validation(self, model, n_splits=2, epochs=50, batch_size=100):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        scores = []

        X = self.X
        y = self.y

        for train_index, val_index in skf.split(self.X, self.y):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            y_train = to_categorical(y_train, num_classes=self.number_of_categories)

            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

            # Evaluate the model
            y_val_pred = np.argmax(model.predict(X_val), axis=1)  # Convert probabilities to class labels
            accuracy = accuracy_score(y_val, y_val_pred)
            scores.append(accuracy)

        print("\nStratified cross-validation scores:", scores)
        print("Average score:", np.mean(scores))

    def ann_cross_validation(self, model, n_splits=2, epochs=50, batch_size=100):
        self.ann_kfold_cross_validation(model, n_splits, epochs, batch_size)
        self.ann_stratified_cross_validation(model, n_splits, epochs, batch_size)

    @staticmethod
    def build_ann(n_neurons=64, activation='relu'):
        model = Sequential()
        # Input layer
        model.add(Dense(n_neurons, activation=activation, input_shape=(24,)))

        model.add(Dense(n_neurons, activation=activation))
        model.add(Dense(n_neurons, activation=activation))
        model.add(Dense(n_neurons, activation=activation))

        # Output layer (example for binary classification)
        model.add(Dense(units=3, activation='softmax'))

        model.compile(optimizer='adam',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

        return model

    def ann_model(self):
        print("#----------------------- #10. Artificial Neural Net Model ------------------------#")

        y_train_tmp = self.y_train

        self.onehot_encode()

        model = KerasClassifier(build_fn=self.build_ann, verbose=0, epochs = 50, batch_size = 100)

        tuned_model = self.hyper_parameter_tuning(model, self.param_grid_ann)
        ann = tuned_model.best_estimator_

        ann.fit(self.X_train, self.y_train)
        y_pred = ann.predict(self.X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        calculate_metrics("Artificial Neural Net", self.y_test, y_pred_classes)
        self.ann_cross_validation(ann)

        self.y_train = y_train_tmp
        
    def driver(self):
        self.run_ann_model()


In [23]:
path = '../data/Processed_Features/'

In [41]:
# Evaluation for W100_O25_Features
evaluate_model = ModelTuningAndEvaluation(path + "W100_O25_Features.csv")

In [25]:
# 1. Logistic Regression on W100_O25_Features
evaluate_model.logistic_regression_model()

#------------------- #1. Logistic Regression Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'C': 100, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score:  0.3515655965120888

#---------- Random Search -----------#
Best parameters:  {'solver': 'liblinear', 'penalty': 'l2', 'max_iter': 100, 'C': 100}
Best score:  0.3515655965120888
Logistic Regression Metrics: 
              precision    recall  f1-score   support

           0       0.26      0.43      0.32      2021
           1       0.37      0.57      0.45      2114
           2       0.26      0.24      0.25      2278
           3       0.22      0.51      0.31      2224
           4       0.32      0.33      0.33      2270
           5       0.32      0.38      0.35      1974
           6       0.27      0.12      0.16      2084
           7       0.16      0.13      0.14      2222
           8       0.60      0.80      0.69      1650
           9       0.87      0.73      0.80     

In [27]:
# 2. Decision Tree Classifier on W100_O25_Features
evaluate_model.decission_tree_classifier_model()

#-------------------- #2. Decission Tree Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 5}
Best score:  0.4975558197912538

#---------- Random Search -----------#
Best parameters:  {'min_samples_split': 2, 'max_depth': 20, 'criterion': 'entropy'}
Best score:  0.5001981767736822
Decission Tree Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.68      0.69      0.69      2021
           1       0.83      0.74      0.79      2114
           2       0.56      0.63      0.60      2278
           3       0.62      0.59      0.61      2224
           4       0.51      0.48      0.49      2270
           5       0.34      0.34      0.34      1974
           6       0.28      0.27      0.27      2084
           7       0.40      0.41      0.41      2222
           8       0.64      0.69      0.67      1650
           9       0.83      0.83      

In [28]:
# 3. Random Forest Classifier on W100_O25_Features
evaluate_model.random_forest_classifier_model()

#-------------------- #3. Random Forest Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'bootstrap': False, 'criterion': 'entropy', 'max_depth': None, 'n_estimators': 500}
Best score:  0.6364116792178623

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 200, 'max_depth': 20, 'criterion': 'entropy', 'bootstrap': False}
Best score:  0.6356189721231339
Random Forest Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.75      0.85      0.80      2021
           1       0.87      0.87      0.87      2114
           2       0.66      0.74      0.70      2278
           3       0.68      0.69      0.69      2224
           4       0.59      0.69      0.64      2270
           5       0.40      0.61      0.49      1974
           6       0.49      0.32      0.39      2084
           7       0.49      0.54      0.51      2222
           8       0.72      0.85      0.78      1650
     

In [29]:
# 4. Gaussian Naive Bayes on W100_O25_Features
evaluate_model.gaussian_naive_bias_classifier_model()

#-------------------- #4. Gaussian Naive Bias Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'var_smoothing': 1e-09}
Best score:  0.2212973972783723

#---------- Random Search -----------#
Best parameters:  {'var_smoothing': 1e-09}
Best score:  0.2212973972783723
Gaussian Naive Bias Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.11      0.88      0.20      2021
           1       0.10      0.02      0.03      2114
           2       0.11      0.07      0.09      2278
           3       0.09      0.04      0.05      2224
           4       0.23      0.13      0.16      2270
           5       0.30      0.18      0.22      1974
           6       0.16      0.07      0.10      2084
           7       0.11      0.06      0.08      2222
           8       0.52      0.65      0.58      1650
           9       0.70      0.70      0.70      1698
          10       0.49      0.57      0.52      1996
   

In [30]:
# 5. Support Vector Classifier on W100_O25_Features
evaluate_model.support_vector_classifier_model()

#-------------------- #5. Support Vector Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
Best score:  0.5768265292641036

#---------- Random Search -----------#
Best parameters:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}
Best score:  0.5768265292641036
Support Vector Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.75      0.78      0.76      2021
           1       0.83      0.85      0.84      2114
           2       0.56      0.72      0.63      2278
           3       0.59      0.68      0.63      2224
           4       0.59      0.61      0.60      2270
           5       0.38      0.41      0.39      1974
           6       0.41      0.33      0.37      2084
           7       0.49      0.46      0.47      2222
           8       0.72      0.78      0.75      1650
           9       0.73      0.88      0.80      1698
          10       0.63      0.

In [31]:
# 6. K-Nearest Neighbors on W100_O25_Features
evaluate_model.knn_classifier_model()

#-------------------- #6. K-Nearest Neighbors Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'distance'}
Best score:  0.49742370194213237

#---------- Random Search -----------#
Best parameters:  {'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'minkowski'}
Best score:  0.49742370194213237
K-Nearest Neighbors Metrics: 
              precision    recall  f1-score   support

           0       0.64      0.73      0.68      2021
           1       0.72      0.79      0.75      2114
           2       0.44      0.65      0.52      2278
           3       0.47      0.73      0.57      2224
           4       0.55      0.50      0.52      2270
           5       0.33      0.40      0.36      1974
           6       0.37      0.17      0.24      2084
           7       0.31      0.38      0.34      2222
           8       0.49      0.86      0.62      1650
           9       0

In [32]:
# 7. AdaBoost Classifier on W100_O25_Features
evaluate_model.ada_boost_classifier_model()

#-------------------- #7. Ada-Boost Classifier Model --------------------#
#-------- Grid Search --------#
Best parameters:  {'estimator': DecisionTreeClassifier(max_depth=3), 'learning_rate': 0.01, 'n_estimators': 100}
Best score:  0.3361078081648831

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 100, 'learning_rate': 0.01, 'estimator': DecisionTreeClassifier(max_depth=3)}
Best score:  0.3361078081648831
Ada-Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.34      0.61      0.44      2021
           1       0.76      0.28      0.41      2114
           2       0.27      0.54      0.36      2278
           3       0.36      0.47      0.41      2224
           4       0.32      0.47      0.38      2270
           5       0.26      0.66      0.37      1974
           6       0.16      0.02      0.04      2084
           7       0.23      0.23      0.23      2222
           8       0.71      0.66      0.69  

In [42]:
# 8. Gradient Boost on W100_O25_Features
evaluate_model.gradient_boost_model()

#-------------------- #9. Gradient Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100, 'random_state': 42}
Best score:  0.6123662306777646

#---------- Random Search -----------#
Best parameters:  {'random_state': 42, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.2}
Best score:  0.6123662306777646
Gradient Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.75      0.82      0.78      2021
           1       0.86      0.84      0.85      2114
           2       0.64      0.70      0.66      2278
           3       0.66      0.68      0.67      2224
           4       0.62      0.64      0.63      2270
           5       0.40      0.52      0.45      1974
           6       0.38      0.33      0.36      2084
           7       0.47      0.52      0.49      2222
           8       0.76      0.82      0.79      1650
     

In [92]:
# 9. XGBoost Classifier on W100_O25_Features
evaluate_model.xg_boost_classifier_model()

-------------------- #8. XG Boost Classifier Model --------------------#
#---------- Grid Search ------------#


KeyboardInterrupt: 

In [ ]:
# 10. Artificial Neural Network on W100_O25_Features
evaluate_model.ann_model()

In [80]:
# Evaluation for W100_O50_Features
evaluate_model = ModelTuningAndEvaluation(path + "W100_O50_Features.csv")

In [35]:
# 1. Logistic Regression on W100_O50_Features
evaluate_model.logistic_regression_model()

#------------------- #1. Logistic Regression Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 100, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score:  0.3527498002302693

#---------- Random Search -----------#
Best parameters:  {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 500, 'C': 100}
Best score:  0.3527498002302693
Logistic Regression Metrics: 
              precision    recall  f1-score   support

           0       0.28      0.39      0.32      3031
           1       0.38      0.63      0.48      3168
           2       0.27      0.27      0.27      3416
           3       0.22      0.58      0.32      3333
           4       0.34      0.33      0.33      3404
           5       0.36      0.38      0.37      2958
           6       0.29      0.14      0.19      3123
           7       0.16      0.15      0.16      3334
           8       0.61      0.78      0.69      2473
           9       0.88      0.75      0.81      2

In [36]:
# 2. Decision Tree Classifier on W100_O50_Features
evaluate_model.decission_tree_classifier_model()

#-------------------- #2. Decission Tree Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}
Best score:  0.5348086262128361

#---------- Random Search -----------#
Best parameters:  {'min_samples_split': 2, 'max_depth': None, 'criterion': 'entropy'}
Best score:  0.533663244033037
Decission Tree Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.72      0.73      0.73      3031
           1       0.82      0.82      0.82      3168
           2       0.62      0.63      0.63      3416
           3       0.64      0.64      0.64      3333
           4       0.54      0.53      0.54      3404
           5       0.35      0.35      0.35      2958
           6       0.32      0.31      0.32      3123
           7       0.45      0.46      0.46      3334
           8       0.68      0.73      0.71      2473
           9       0.84      0

In [ ]:
# 3. Random Forest Classifier on W100_O50_Features
evaluate_model.random_forest_classifier_model()

In [81]:
# 4. Gaussian Naive Bayes on W100_O50_Features
evaluate_model.gaussian_naive_bias_classifier_model()

#-------------------- #4. Gaussian Naive Bias Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.24656051018036984

#---------- Random Search -----------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.24656051018036984

Gaussian Naive Bias Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.16      0.81      0.27      3031
           1       0.12      0.07      0.09      3168
           2       0.11      0.05      0.07      3416
           3       0.19      0.50      0.27      3333
           4       0.23      0.13      0.16      3404
           5       0.30      0.17      0.22      2958
           6       0.14      0.07      0.09      3123
           7       0.13      0.08      0.10      3334
           8       0.53      0.62      0.57      2473
           9       0.80      0.71      0.76      2536
          10       0.47      0.58      0.52   

In [82]:
# 5. Support Vector Classifier on W100_O50_Features
evaluate_model.support_vector_classifier_model()

#-------------------- #5. Support Vector Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
Best score:  0.5836261324575477

#---------- Random Search -----------#
Best parameters:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}
Best score:  0.5836261324575477

Support Vector Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.79      0.81      0.80      3031
           1       0.86      0.87      0.86      3168
           2       0.61      0.74      0.67      3416
           3       0.61      0.71      0.66      3333
           4       0.61      0.65      0.63      3404
           5       0.43      0.50      0.46      2958
           6       0.46      0.35      0.40      3123
           7       0.52      0.48      0.50      3334
           8       0.75      0.81      0.78      2473
           9       0.89      0.88      0.88      2536
          10       0.62 

In [83]:
# 6. K-Nearest Neighbors on W100_O50_Features
evaluate_model.knn_classifier_model()

#-------------------- #6. K-Nearest Neighbors Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'distance'}
Best score:  0.5176236532579929

#---------- Random Search -----------#
Best parameters:  {'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'minkowski'}
Best score:  0.5176236532579929

K-Nearest Neighbors Metrics: 
              precision    recall  f1-score   support

           0       0.67      0.66      0.67      3031
           1       0.71      0.84      0.77      3168
           2       0.46      0.71      0.56      3416
           3       0.54      0.67      0.60      3333
           4       0.55      0.53      0.54      3404
           5       0.36      0.40      0.38      2958
           6       0.43      0.20      0.27      3123
           7       0.34      0.48      0.40      3334
           8       0.50      0.86      0.64      2473
           9   

In [84]:
# 7. AdaBoost Classifier on W100_O50_Features
evaluate_model.ada_boost_classifier_model()

#-------------------- #7. Ada-Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'estimator': DecisionTreeClassifier(max_depth=1), 'learning_rate': 0.5, 'n_estimators': 100}
Best score:  0.2658628162888713

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 100, 'learning_rate': 0.5, 'estimator': DecisionTreeClassifier(max_depth=1)}
Best score:  0.2658628162888713

Ada-Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.21      0.30      0.24      3031
           1       0.22      0.49      0.30      3168
           2       0.23      0.21      0.22      3416
           3       0.11      0.01      0.02      3333
           4       0.26      0.29      0.27      3404
           5       0.35      0.34      0.34      2958
           6       0.18      0.05      0.08      3123
           7       0.15      0.34      0.20      3334
           8       0.37      0.59      0

In [ ]:
# 8. Gradient Boost on W100_O50_Features
evaluate_model.gradient_boost_model()

In [85]:
# 9. XGBoost Classifier on W100_O50_Features
evaluate_model.xg_boost_classifier_model()

-------------------- #8. XG Boost Classifier Model --------------------#
#---------- Grid Search ------------#


KeyboardInterrupt: 

In [ ]:
# 10. Artificial Neural Network on W100_O50_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W200_O25_Features
evaluate_model = ModelTuningAndEvaluation(path + "W200_O25_Features.csv")

In [ ]:
# 1. Logistic Regression on W200_O25_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W200_O25_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W200_O25_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W200_O25_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W200_O25_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W200_O25_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W200_O25_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W200_O25_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W200_O25_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W200_O25_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W200_O50_Features
evaluate_model = ModelTuningAndEvaluation(path + "W200_O50_Features.csv")

In [ ]:
# 1. Logistic Regression on W200_O50_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W200_O50_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W200_O50_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W200_O50_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W200_O50_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W200_O50_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W200_O50_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W200_O50_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W200_O50_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W200_O50_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W300_O25_Features
evaluate_model = ModelTuningAndEvaluation(path + "W300_O25_Features.csv")

In [ ]:
# 1. Logistic Regression on W300_O25_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W300_O25_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W300_O25_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W300_O25_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W300_O25_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W300_O25_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W300_O25_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W300_O25_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W300_O25_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W300_O25_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W300_O50_Features
evaluate_model = ModelTuningAndEvaluation(path + "W300_O50_Features.csv")

In [ ]:
# 1. Logistic Regression on W300_O50_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W300_O50_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W300_O50_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W300_O50_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W300_O50_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W300_O50_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W300_O50_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W300_O50_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W300_O50_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W300_O50_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W400_O25_Features
evaluate_model = ModelTuningAndEvaluation(path + "W400_O25_Features.csv")

In [ ]:
# 1. Logistic Regression on W400_O25_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W400_O25_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W400_O25_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W400_O25_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W400_O25_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W400_O25_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W400_O25_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W400_O25_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W400_O25_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W400_O25_Features
evaluate_model.ann_model()

In [ ]:
# Evaluation for W400_O50_Features
evaluate_model = ModelTuningAndEvaluation(path + "W400_O50_Features.csv")

In [ ]:
# 1. Logistic Regression on W400_O50_Features
evaluate_model.logistic_regression_model()

In [ ]:
# 2. Decision Tree Classifier on W400_O50_Features
evaluate_model.decission_tree_classifier_model()

In [ ]:
# 3. Random Forest Classifier on W400_O50_Features
evaluate_model.random_forest_classifier_model()

In [ ]:
# 4. Gaussian Naive Bayes on W400_O50_Features
evaluate_model.gaussian_naive_bias_classifier_model()

In [ ]:
# 5. Support Vector Classifier on W400_O50_Features
evaluate_model.support_vector_classifier_model()

In [ ]:
# 6. K-Nearest Neighbors on W400_O50_Features
evaluate_model.knn_classifier_model()

In [ ]:
# 7. AdaBoost Classifier on W400_O50_Features
evaluate_model.ada_boost_classifier_model()

In [ ]:
# 8. Gradient Boost on W400_O50_Features
evaluate_model.gradient_boost_model()

In [ ]:
# 9. XGBoost Classifier on W400_O50_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W400_O50_Features
evaluate_model.ann_model()

In [103]:
# Evaluation for W500_O25_Features
evaluate_model = ModelTuningAndEvaluation(path + "W500_O25_Features.csv")

In [104]:
# 1. Logistic Regression on W500_O25_Features
evaluate_model.logistic_regression_model()

#------------------- #1. Logistic Regression Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 10, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score:  0.4216682958618443

#---------- Random Search -----------#
Best parameters:  {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 100, 'C': 10}
Best score:  0.4216682958618443

Logistic Regression Metrics: 
              precision    recall  f1-score   support

           0       0.28      0.38      0.32       397
           1       0.43      0.65      0.51       414
           2       0.35      0.34      0.34       447
           3       0.28      0.53      0.37       437
           4       0.33      0.29      0.31       448
           5       0.48      0.49      0.49       388
           6       0.42      0.38      0.39       410
           7       0.20      0.21      0.20       439
           8       0.60      0.70      0.64       324
           9       0.89      0.80      0.84       3

In [105]:
# 2. Decision Tree Classifier on W500_O25_Features
evaluate_model.decission_tree_classifier_model()

#-------------------- #2. Decission Tree Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 5}
Best score:  0.42973281199087654

#---------- Random Search -----------#
Best parameters:  {'min_samples_split': 5, 'max_depth': 20, 'criterion': 'entropy'}
Best score:  0.43041028565222117

Decission Tree Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.57      0.57      0.57       397
           1       0.67      0.71      0.69       414
           2       0.46      0.43      0.44       447
           3       0.46      0.55      0.50       437
           4       0.32      0.31      0.31       448
           5       0.37      0.42      0.39       388
           6       0.25      0.31      0.28       410
           7       0.26      0.30      0.28       439
           8       0.57      0.64      0.60       324
           9       0.85      0

In [106]:
# 3. Random Forest Classifier on W500_O25_Features
evaluate_model.random_forest_classifier_model()

#-------------------- #3. Random Forest Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 100}
Best score:  0.5635861301183882

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 50, 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}
Best score:  0.557535027696318

Random Forest Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.63      0.69      0.65       397
           1       0.67      0.82      0.73       414
           2       0.60      0.64      0.62       447
           3       0.57      0.58      0.58       437
           4       0.47      0.60      0.53       448
           5       0.47      0.69      0.56       388
           6       0.40      0.48      0.43       410
           7       0.41      0.46      0.44       439
           8       0.63      0.84      0.72       324
       

In [107]:
# 4. Gaussian Naive Bayes on W500_O25_Features
evaluate_model.gaussian_naive_bias_classifier_model()

#-------------------- #4. Gaussian Naive Bias Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.2837976539589443

#---------- Random Search -----------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.2837976539589443

Gaussian Naive Bias Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.30      0.13      0.18       397
           1       0.29      0.04      0.07       414
           2       0.13      0.06      0.08       447
           3       0.16      0.89      0.27       437
           4       0.32      0.23      0.27       448
           5       0.36      0.28      0.32       388
           6       0.28      0.31      0.30       410
           7       0.17      0.15      0.16       439
           8       0.45      0.61      0.52       324
           9       0.76      0.72      0.74       339
          10       0.60      0.59      0.59     

In [108]:
# 5. Support Vector Classifier on W500_O25_Features
evaluate_model.support_vector_classifier_model()

#-------------------- #5. Support Vector Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
Best score:  0.534646193113935

#---------- Random Search -----------#
Best parameters:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}
Best score:  0.534646193113935

Support Vector Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.71      0.76      0.73       397
           1       0.79      0.80      0.80       414
           2       0.56      0.60      0.58       447
           3       0.59      0.62      0.60       437
           4       0.58      0.53      0.55       448
           5       0.42      0.53      0.47       388
           6       0.41      0.49      0.45       410
           7       0.44      0.43      0.44       439
           8       0.66      0.77      0.71       324
           9       0.73      0.87      0.79       339
          10       0.76   

In [109]:
# 6. K-Nearest Neighbors on W500_O25_Features
evaluate_model.knn_classifier_model()

#-------------------- #6. K-Nearest Neighbors Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'distance'}
Best score:  0.3873845986749213

#---------- Random Search -----------#
Best parameters:  {'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'minkowski'}
Best score:  0.3873845986749213

K-Nearest Neighbors Metrics: 
              precision    recall  f1-score   support

           0       0.41      0.47      0.44       397
           1       0.49      0.77      0.60       414
           2       0.39      0.64      0.48       447
           3       0.43      0.67      0.52       437
           4       0.40      0.25      0.31       448
           5       0.39      0.37      0.38       388
           6       0.26      0.41      0.32       410
           7       0.28      0.29      0.28       439
           8       0.34      0.70      0.46       324
           9   

In [110]:
# 7. AdaBoost Classifier on W500_O25_Features
evaluate_model.ada_boost_classifier_model()

#-------------------- #7. Ada-Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'estimator': DecisionTreeClassifier(max_depth=1), 'learning_rate': 0.5, 'n_estimators': 50}
Best score:  0.24207668078635822

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 50, 'learning_rate': 0.5, 'estimator': DecisionTreeClassifier(max_depth=1)}
Best score:  0.24207668078635822

Ada-Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       397
           1       0.24      0.36      0.29       414
           2       0.09      0.01      0.02       447
           3       0.18      0.91      0.29       437
           4       0.24      0.27      0.25       448
           5       0.36      0.52      0.43       388
           6       0.22      0.10      0.13       410
           7       0.00      0.00      0.00       439
           8       0.30      0.40      0

In [ ]:
# 8. Gradient Boost on W500_O25_Features
evaluate_model.gradient_boost_model()

#-------------------- #9. Gradient Boost Classifier Model --------------------#
#---------- Grid Search ------------#


In [ ]:
# 9. XGBoost Classifier on W500_O25_Features
evaluate_model.xg_boost_classifier_model()

In [ ]:
# 10. Artificial Neural Network on W500_O25_Features
evaluate_model.ann_model()

In [95]:
# Evaluation for W500_O50_Features
evaluate_model = ModelTuningAndEvaluation(path + "W500_O50_Features.csv")

In [75]:
# 1. Logistic Regression on W500_O50_Features
evaluate_model.logistic_regression_model()

#------------------- #1. Logistic Regression Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 10, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Best score:  0.437269656864363

#---------- Random Search -----------#
Best parameters:  {'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 500, 'C': 10}
Best score:  0.437269656864363
Logistic Regression Metrics: 
              precision    recall  f1-score   support

           0       0.38      0.43      0.41       591
           1       0.43      0.66      0.52       619
           2       0.36      0.38      0.37       667
           3       0.29      0.56      0.38       651
           4       0.39      0.34      0.36       667
           5       0.48      0.54      0.51       579
           6       0.44      0.45      0.45       610
           7       0.27      0.19      0.22       652
           8       0.68      0.70      0.69       484
           9       0.90      0.84      0.87       502


In [96]:
# 2. Decision Tree Classifier on W500_O50_Features
evaluate_model.decission_tree_classifier_model()

#-------------------- #2. Decission Tree Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 5}
Best score:  0.4787757327280084

#---------- Random Search -----------#
Best parameters:  {'min_samples_split': 2, 'max_depth': 50, 'criterion': 'entropy'}
Best score:  0.4851119643357011

Decission Tree Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.65      0.64      0.65       591
           1       0.73      0.69      0.71       619
           2       0.55      0.55      0.55       667
           3       0.51      0.54      0.52       651
           4       0.42      0.39      0.40       667
           5       0.45      0.46      0.45       579
           6       0.38      0.38      0.38       610
           7       0.38      0.43      0.40       652
           8       0.66      0.68      0.67       484
           9       0.88      0.85

In [97]:
# 3. Random Forest Classifier on W500_O50_Features
evaluate_model.random_forest_classifier_model()

#-------------------- #3. Random Forest Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 100}
Best score:  0.598825532684736

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 100, 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}
Best score:  0.6096540284302256

Random Forest Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.76      0.81      0.79       591
           1       0.83      0.84      0.83       619
           2       0.59      0.76      0.67       667
           3       0.66      0.69      0.67       651
           4       0.58      0.60      0.59       667
           5       0.54      0.73      0.62       579
           6       0.49      0.55      0.52       610
           7       0.49      0.54      0.52       652
           8       0.76      0.84      0.80       484
      

In [98]:
# 4. Gaussian Naive Bayes on W500_O50_Features
evaluate_model.gaussian_naive_bias_classifier_model()

#-------------------- #4. Gaussian Naive Bias Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.2847441487495615

#---------- Random Search -----------#
Best parameters:  {'var_smoothing': 0.0001}
Best score:  0.2847441487495615

Gaussian Naive Bias Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.14      0.86      0.25       591
           1       0.31      0.06      0.10       619
           2       0.21      0.16      0.18       667
           3       0.19      0.03      0.05       651
           4       0.29      0.22      0.25       667
           5       0.45      0.28      0.35       579
           6       0.32      0.41      0.36       610
           7       0.14      0.10      0.12       652
           8       0.52      0.60      0.55       484
           9       0.71      0.73      0.72       502
          10       0.57      0.66      0.61     

In [99]:
# 5. Support Vector Classifier on W500_O50_Features
evaluate_model.support_vector_classifier_model()

#-------------------- #5. Support Vector Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'}
Best score:  0.5762572533258035

#---------- Random Search -----------#
Best parameters:  {'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}
Best score:  0.5762572533258035

Support Vector Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.77      0.77      0.77       591
           1       0.83      0.84      0.83       619
           2       0.62      0.76      0.69       667
           3       0.63      0.65      0.64       651
           4       0.61      0.65      0.63       667
           5       0.51      0.59      0.54       579
           6       0.52      0.47      0.50       610
           7       0.54      0.49      0.51       652
           8       0.76      0.83      0.79       484
           9       0.87      0.90      0.89       502
          10       0.71 

In [100]:
# 6. K-Nearest Neighbors on W500_O50_Features
evaluate_model.knn_classifier_model()

#-------------------- #6. K-Nearest Neighbors Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'distance'}
Best score:  0.4300478074695045

#---------- Random Search -----------#
Best parameters:  {'weights': 'distance', 'p': 1, 'n_neighbors': 100, 'metric': 'minkowski'}
Best score:  0.4300478074695045

K-Nearest Neighbors Metrics: 
              precision    recall  f1-score   support

           0       0.57      0.69      0.63       591
           1       0.58      0.72      0.65       619
           2       0.43      0.67      0.52       667
           3       0.43      0.73      0.54       651
           4       0.52      0.32      0.40       667
           5       0.42      0.43      0.42       579
           6       0.28      0.40      0.33       610
           7       0.34      0.39      0.36       652
           8       0.43      0.76      0.55       484
           9   

In [101]:
# 7. AdaBoost Classifier on W500_O50_Features
evaluate_model.ada_boost_classifier_model()

#-------------------- #7. Ada-Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'estimator': DecisionTreeClassifier(max_depth=1), 'learning_rate': 0.5, 'n_estimators': 50}
Best score:  0.2125354338793726

#---------- Random Search -----------#
Best parameters:  {'n_estimators': 50, 'learning_rate': 0.5, 'estimator': DecisionTreeClassifier(max_depth=1)}
Best score:  0.2125354338793726

Ada-Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.37      0.38      0.38       591
           1       0.00      0.00      0.00       619
           2       0.16      0.31      0.21       667
           3       0.17      0.29      0.22       651
           4       0.20      0.31      0.25       667
           5       0.34      0.58      0.43       579
           6       0.14      0.10      0.12       610
           7       0.13      0.09      0.11       652
           8       0.27      0.08      0.1

In [77]:
# 8. Gradient Boost on W500_O50_Features
evaluate_model.gradient_boost_model()

#-------------------- #9. Gradient Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'random_state': 42}
Best score:  0.5879945921696476

#---------- Random Search -----------#
Best parameters:  {'random_state': 42, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1}
Best score:  0.5879945921696476
Gradient Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.74      0.69      0.72       591
           1       0.80      0.84      0.82       619
           2       0.66      0.71      0.69       667
           3       0.62      0.64      0.63       651
           4       0.62      0.64      0.63       667
           5       0.57      0.74      0.65       579
           6       0.45      0.51      0.48       610
           7       0.49      0.53      0.51       652
           8       0.81      0.82      0.81       484
     

In [64]:
# 9. XGBoost Classifier on W500_O50_Features
evaluate_model.xg_boost_classifier_model()

-------------------- #8. XG Boost Classifier Model --------------------#
#---------- Grid Search ------------#
Best parameters:  {'gamma': 0, 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 0.8}
Best score:  0.628156655946352

#---------- Random Search -----------#
Best parameters:  {'subsample': 0.8, 'n_estimators': 100, 'learning_rate': 0.1, 'gamma': 0}
Best score:  0.628156655946352
XG Boost Classifier Metrics: 
              precision    recall  f1-score   support

           0       0.81      0.76      0.78       591
           1       0.85      0.89      0.87       619
           2       0.67      0.77      0.72       667
           3       0.69      0.69      0.69       651
           4       0.63      0.66      0.65       667
           5       0.64      0.77      0.70       579
           6       0.50      0.57      0.53       610
           7       0.53      0.54      0.54       652
           8       0.80      0.86      0.83       484
           9       0.92      0.9

In [93]:
# 10. Artificial Neural Network on W500_O50_Features
evaluate_model.ann_model()

#----------------------- #10. Artificial Neural Net Model ------------------------#
(11348, 15)
#---------- Grid Search ------------#


ValueError: 
All the 24 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nahid\miniconda3\envs\tf_env\lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nahid\miniconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py", line 1501, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "C:\Users\nahid\miniconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py", line 770, in fit
    self._fit(
  File "C:\Users\nahid\miniconda3\envs\tf_env\lib\site-packages\scikeras\wrappers.py", line 938, in _fit
    self._fit_keras_model(
  File "C:\Users\nahid\miniconda3\envs\tf_en